In [ ]:
import numpy as np
from numpy import pi as π
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import tqdm
import firedrake
from firedrake import (
    max_value, Constant, exp, sin, cos, inner, grad, dx, ds, dS, avg, jump
)
import irksome
from irksome import Dt

In [ ]:
mesh = firedrake.UnitDiskMesh(4)
n = firedrake.FacetNormal(mesh)
fig, ax = plt.subplots()
ax.set_aspect("equal")
firedrake.triplot(mesh, axes=ax);

In [ ]:
dg0 = firedrake.FiniteElement("DG", "triangle", 0)
cg1 = firedrake.FiniteElement("CG", "triangle", 1)
Q = firedrake.FunctionSpace(mesh, dg0)
S = firedrake.FunctionSpace(mesh, cg1)

In [ ]:
x = firedrake.SpatialCoordinate(mesh)
h_expr = Constant(1.0)
r = Constant(0.125)
ξ = Constant((0.25, 0.25))
E_expr = exp(-inner(x - ξ, x - ξ) / r**2)

In [ ]:
t = firedrake.Constant(0.0)
ξ_1 = Constant((+0.5, 0.0))
ξ_2 = Constant((-0.5, 0.0))
R = Constant(0.25)
a_0 = Constant(+0.1)
accumulation = a_0 * max_value(0, cos(t)) * exp(-inner(x - ξ_1, x - ξ_1) / R**2)
ablation = a_0 * max_value(0, sin(t)) * exp(-inner(x - ξ_2, x - ξ_2) / R**2)
a = accumulation - ablation

In [ ]:
u = firedrake.as_vector((-x[1], x[0]))

In [ ]:
Z = Q * Q
z = firedrake.Function(Z)
z.subfunctions[0].project(h_expr)
z.subfunctions[1].project(E_expr)

h, E = firedrake.split(z)
η, ϕ = firedrake.TestFunctions(Z)

In [ ]:
F_cells = h * (Dt(E) * ϕ - inner(E * u, grad(ϕ))) * dx
f = E * firedrake.max_value(0, inner(u, n))
F_facets = avg(h) * jump(f) * jump(ϕ) * dS

F_E = F_cells + F_facets

In [ ]:
F_cells = (Dt(h) * η - inner(h * u, grad(η)) - a * η) * dx
f = h * firedrake.max_value(0, inner(u, n))
F_facets = jump(f) * jump(η) * dS

F_h = F_cells + F_facets

In [ ]:
F = F_E + F_h

In [ ]:
method = irksome.BackwardEuler()
final_time = 4 * π
num_steps = 256
dt = Constant(final_time / num_steps)
solver = irksome.TimeStepper(F, method, t, dt, z)

In [ ]:
zs = [z.copy(deepcopy=True)]
for step in tqdm.trange(num_steps):
    solver.advance()
    zs.append(z.copy(deepcopy=True))
    t.assign(t + dt)

In [ ]:
hmin = np.array([z.subfunctions[0].dat.data_ro.min()]).min()
hmax = np.array([z.subfunctions[0].dat.data_ro.max()]).max()
hmin, hmax

In [ ]:
Emin = np.array([z.subfunctions[1].dat.data_ro.min() for z in zs]).min()
Emax = np.array([z.subfunctions[1].dat.data_ro.max() for z in zs]).max()
Emin, Emax

In [ ]:
%%capture

fig, axes = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True)
for ax in axes:
    ax.set_aspect("equal")
    ax.axis("off")

h, E = zs[0].subfunctions
kw = {"num_sample_points": 1, "shading": "gouraud"}
colors_h = firedrake.tripcolor(h, vmin=hmin, vmax=hmax, axes=axes[0], **kw)
colors_E = firedrake.tripcolor(E, vmin=Emin, vmax=Emax, axes=axes[1], **kw)
fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=1)

def animate(z):
    h, E = z.subfunctions
    colors_h.set_array(fn_plotter(h))
    colors_E.set_array(fn_plotter(E))

animation = FuncAnimation(fig, animate, zs, interval=1e3/24)

In [ ]:
HTML(animation.to_html5_video())